### This notebook preprocesses raw ridership datasets.

In [1]:
import pandas as pd
import numpy as np
import yaml
from glob import glob
import os
from pathlib import Path
from datetime import date, timedelta, datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pathlib import Path
import re
import hashlib

In [2]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## Agency Dataset Preprocess

This section preprocess the raw datasets so that they can be aggregated into one dataset with pre-specified schema.

Some datasets may not need pre-process.

In [3]:
def get_fiscal_year_range(fiscal_year: int, start_month: int):
    # given fiscal year and start month, return the start and end date of the fiscal year
    
    start_year = fiscal_year - 1 if start_month != 1 else fiscal_year
    start_date = date(start_year, start_month, 1)
    
    # compute end date
    if start_month == 1:
        end_date = date(fiscal_year, 12, 31)
    else:
        end_month = (start_month-1)
        next_month = date(fiscal_year, start_month, 1)
        end_date = next_month - timedelta(days=1)
    
    return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")
        

In [4]:
get_fiscal_year_range(2025, 7)

('2024-07-01', '2025-06-30')

In [5]:
# get holidays
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2020-01-01', end='2025-12-31')

def get_day_type(date):
    if date in holidays:
        return "holiday"
    elif date.weekday() < 5:
        return "weekday"
    else:
        return "weekend"

In [6]:
holidays

DatetimeIndex(['2020-01-01', '2020-01-20', '2020-02-17', '2020-05-25',
               '2020-07-03', '2020-09-07', '2020-10-12', '2020-11-11',
               '2020-11-26', '2020-12-25', '2021-01-01', '2021-01-18',
               '2021-02-15', '2021-05-31', '2021-06-18', '2021-07-05',
               '2021-09-06', '2021-10-11', '2021-11-11', '2021-11-25',
               '2021-12-24', '2021-12-31', '2022-01-17', '2022-02-21',
               '2022-05-30', '2022-06-20', '2022-07-04', '2022-09-05',
               '2022-10-10', '2022-11-11', '2022-11-24', '2022-12-26',
               '2023-01-02', '2023-01-16', '2023-02-20', '2023-05-29',
               '2023-06-19', '2023-07-04', '2023-09-04', '2023-10-09',
               '2023-11-10', '2023-11-23', '2023-12-25', '2024-01-01',
               '2024-01-15', '2024-02-19', '2024-05-27', '2024-06-19',
               '2024-07-04', '2024-09-02', '2024-10-14', '2024-11-11',
               '2024-11-28', '2024-12-25', '2025-01-01', '2025-01-20',
      

#### Foothill Transit

- No header/column names provided. Need to do some exploration and compare to the GTFS data to identify route id and stop ids.
- Data is at stop event level. Need to aggregate.
- Raw dataset is large and won't be pushed to git repo yet.

In [37]:
# df_foothill_stops = pd.read_csv("GTFS/Foothill Transit/stops.txt", delimiter=",")
# df_foothill_stops.head()

In [38]:
# df_foothill_trips = pd.read_csv("GTFS/Foothill Transit/trips.txt", delimiter=",")
# df_foothill_trips.head()

In [39]:
# df_foothill_routes = pd.read_csv("GTFS/Foothill Transit/routes.txt", delimiter=",")
# df_foothill_routes.head()

In [40]:
# df_foothill_stop_times = pd.read_csv("GTFS/Foothill Transit/stop_times.txt", delimiter=",")
# df_foothill_stop_times.head()

In [41]:
# df_foothill_stop_times["stop_code"] = df_foothill_stop_times["stop_id"].str.extract(r"^\s*(\d+)\s*[--]")
# df_foothill_stop_times.head()

In [36]:
# df_foothill_stop_times[df_foothill_stop_times["stop_code"] == '2158'][["arrival_time","departure_time"]].drop_duplicates().reset_index(drop=True).sort_values(by="arrival_time", ascending=True)

In [28]:
column_names=["unknown_1", "date", "unknown_2", "block_id", "route_short_name", "unknown_3", "direction", "stop_code", "unknown_4",
              "unknown_5", "unknown_6", "stop_lat", "stop_lon", "boardings", "alightings", "max_load"]

raw_foothill = pd.read_csv("transit_agency_ridership_raw_datasets/Foothill Transit/Foothill Transit FY2025 Ridership Data.csv", header=None, names=column_names)
raw_foothill.head()

,unknown_1,date,unknown_2,block_id,route_short_name,unknown_3,direction,stop_code,unknown_4,unknown_5,unknown_6,stop_lat,stop_lon,boardings,alightings,max_load
0,2024-07-01 03:06:40.000,2024-07-01 00:00:00.000,2922,10005,707,(X) 0300-W,W,2028,1,03:00:00.0000000,04:16:00.0000000,34.094501,-117.694932,22,0,19
1,2024-07-01 03:22:36.000,2024-07-01 00:00:00.000,2922,10005,707,(X) 0300-W,W,2158,2,03:00:00.0000000,04:16:00.0000000,34.059396,-117.751410,4,4,22
2,2024-07-01 03:39:00.000,2024-07-01 00:00:00.000,2922,10005,707,(X) 0300-W,W,494,3,03:00:00.0000000,04:16:00.0000000,34.072337,-117.906536,2,0,24
3,2024-07-01 03:44:07.000,2024-07-01 00:00:00.000,2922,10005,707,(X) 0300-W,W,2924,5,03:00:00.0000000,04:16:00.0000000,34.068646,-117.929815,1,1,24
4,2024-07-01 04:00:07.000,2024-07-01 00:00:00.000,2211,90001,699,(X) 0400-W,W,2028,1,04:00:00.0000000,05:06:00.0000000,34.094501,-117.694932,7,1,6


In [30]:
raw_foothill["date"] = pd.to_datetime(raw_foothill["date"]).dt.floor('D')

,unknown_1,date,unknown_2,block_id,route_short_name,unknown_3,direction,stop_code,unknown_4,unknown_5,unknown_6,stop_lat,stop_lon,boardings,alightings,max_load
0,2024-07-01 03:06:40.000,2024-07-01,2922,10005,707,(X) 0300-W,W,2028,1,03:00:00.0000000,04:16:00.0000000,34.094501,-117.694932,22,0,19
1,2024-07-01 03:22:36.000,2024-07-01,2922,10005,707,(X) 0300-W,W,2158,2,03:00:00.0000000,04:16:00.0000000,34.059396,-117.751410,4,4,22
2,2024-07-01 03:39:00.000,2024-07-01,2922,10005,707,(X) 0300-W,W,494,3,03:00:00.0000000,04:16:00.0000000,34.072337,-117.906536,2,0,24
3,2024-07-01 03:44:07.000,2024-07-01,2922,10005,707,(X) 0300-W,W,2924,5,03:00:00.0000000,04:16:00.0000000,34.068646,-117.929815,1,1,24
4,2024-07-01 04:00:07.000,2024-07-01,2211,90001,699,(X) 0400-W,W,2028,1,04:00:00.0000000,05:06:00.0000000,34.094501,-117.694932,7,1,6


In [53]:
raw_foothill_export = raw_foothill.groupby(by=["date", "route_short_name", "direction", "stop_code", "stop_lat", "stop_lon"], as_index=False).agg(
                                                                                                                                                  boardings = ("boardings", "sum"),
                                                                                                                                                  alightings = ("alightings", "sum"))
raw_foothill_export["start_date"] = raw_foothill_export["date"]
raw_foothill_export["end_date"] = raw_foothill_export["date"]

raw_foothill_export["day_type"] = raw_foothill_export["date"].apply(get_day_type)

raw_foothill_export.to_excel("transit_agency_ridership_preprocessed_datasets/foothill_transit_ridership.xlsx")
raw_foothill_export.head()

,date,route_short_name,direction,stop_code,stop_lat,stop_lon,boardings,alightings,start_date,end_date,day_type
0,2024-07-01,178,E,23,34.034964,-117.919263,1,0,2024-07-01,2024-07-01,weekday
1,2024-07-01,178,E,555,34.030813,-117.914021,56,40,2024-07-01,2024-07-01,weekday
2,2024-07-01,178,E,603,34.029240,-117.910251,6,2,2024-07-01,2024-07-01,weekday
3,2024-07-01,178,E,609,34.035615,-117.926121,7,7,2024-07-01,2024-07-01,weekday
4,2024-07-01,178,E,614,34.027174,-117.906451,5,1,2024-07-01,2024-07-01,weekday


In [42]:
# t_df_foothill = raw_foothill.groupby(by=["date", "route_short_name", "direction", "stop_code"])[["stop_lat", "stop_lon"]].apply(lambda x: len(x.drop_duplicates())).reset_index(name="num_loc")
# t_df_foothill.sort_values(by="num_loc", ascending=False)

#### BART

- Reformat data from wide to long format.
- Join raw data and station crosswalk tables to get full station name.

In [7]:
raw_bart = pd.read_excel("transit_agency_ridership_raw_datasets/BART/Entries and Exits by Station Oct 24 to Sept 25.xlsx", sheet_name="Daily Raw Data", header=[0,1])
raw_bart.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  FY               Year              Month         Month Name   
0               FY25               2024                 10                Oct   
1               FY25               2024                 10                Oct   
2               FY25               2024                 10                Oct   
3               FY25               2024                 10                Oct   
4               FY25               2024                 10                Oct   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Exits              \
                Date        Day of Week           Day Type    RM    EN    EP   
0         2024-10-01            Tuesday            Weekday  2581  4256  2343   
1         2024-10-02          Wednesday            Weekday  2664  4142  2289   
2         2024-10-03           Thursday            Weekday  2445  4158  2235   
3         2024-10-04             Friday            Weekday  1798  3587  1923   
4         2024-10-05           Saturday           Saturday    21  2213  1327   

   ... Entries                                                      
   ...      SB    SO    MB    WD   OA    WS   PC    AN    ML    BE  
0  ...    1667  4484  2036  1467  598  1415  614  2293  1398  1726  
1  ...    1685  4084  2060  1424  536  1354  542  2258  1402  1672  
2  ...    1674  4783  2152  1419  598  1476  614  2094  1572  1932  
3  ...    1334  4851  1914   988  746  1084  541  1671  1154  1503  
4  ...     791  3699  1207   626  425   584  224   717   944  1296  

[5 rows x 107 columns]

In [8]:
# get level 1 and level 2 headers
top_headers = raw_bart.columns.get_level_values(0)
bottom_headers  = raw_bart.columns.get_level_values(1)

# identify metadata columns and ridership columns (entries/exits)
meta_cols = [c for c, top in zip(raw_bart.columns, top_headers) if "Unnamed" in str(top)]
ridership_cols = [c for c, top in zip(raw_bart.columns, top_headers) if ("Exits"  in str(top)) or ("Entries" in str(top))]
# raw_bart.columns = []

# melt ridership columns
t_df_ridership = raw_bart.melt(id_vars=meta_cols, value_vars=ridership_cols, var_name=["type", "Station"], value_name="ridership")

# pivot table
t_df_ridership_pivot = t_df_ridership.pivot_table(index=meta_cols+["Station"],
                                                  columns="type",
                                                  values="ridership",
                                                  aggfunc="sum")

t_df_ridership_pivot = t_df_ridership_pivot.reset_index()
# t_df_ridership_pivot.drop(columns="type", axis=1, inplace=True)
t_df_ridership_pivot.columns = [bottom_headers[i] if i < len(meta_cols) else t_df_ridership_pivot.columns[i] for i in range(len(t_df_ridership_pivot.columns))]
t_df_ridership_pivot.head()

,FY,Year,Month,Month Name,Date,Day of Week,Day Type,Station,Entries,Exits
0,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,12,5946,5918
1,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,16,6259,6015
2,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,19,5573,5432
3,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,24,6303,6138
4,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,AN,2293,2239


In [9]:
raw_bart_station = pd.read_excel("transit_agency_ridership_raw_datasets/BART/Entries and Exits by Station Oct 24 to Sept 25.xlsx", sheet_name="Station Crosswalk")
raw_bart_station.head(2)

,Two-Letter Station Code,Station Name
0,RM,Richmond
1,EN,El Cerrito Del Norte


In [10]:
raw_bart_station.columns = ["Station Code", "Station Name"]

In [11]:
# add station name
raw_bart_export = pd.merge(t_df_ridership_pivot, raw_bart_station, how="left", left_on="Station", right_on="Station Code")
raw_bart_export = raw_bart_export.drop(columns="Station Code")

# add date range
raw_bart_export["start_date"] = raw_bart_export["Date"]
raw_bart_export["end_date"] = raw_bart_export["Date"]

raw_bart_export.to_excel("transit_agency_ridership_preprocessed_datasets/bart_ridership.xlsx")
raw_bart_export.head()

,FY,Year,Month,Month Name,Date,Day of Week,Day Type,Station,Entries,Exits,Station Name,start_date,end_date
0,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,12,5946,5918,12th Street / Oakland City Center,2024-10-01,2024-10-01
1,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,16,6259,6015,16th Street Mission,2024-10-01,2024-10-01
2,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,19,5573,5432,19th Street / Oakland,2024-10-01,2024-10-01
3,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,24,6303,6138,24th Street Mission,2024-10-01,2024-10-01
4,FY25,2024,10,Oct,2024-10-01,Tuesday,Weekday,AN,2293,2239,Antioch,2024-10-01,2024-10-01


#### Big Blue Bus (City of Santa Monica)

Add start and end date of service period/aggregation period.

**Note: Each stop id can have more than 1 record.**

In [12]:
raw_big_blue_bus = pd.read_excel("transit_agency_ridership_raw_datasets/Big Blue Bus/CADOT_REQ_AVG_RIDERSHIP BY ROUTE AND STOP (DATAVIEW).xlsx")
raw_big_blue_bus.head(3)

,SERVICE_PERIOD,SERVICE_DAY,ROUTE_NUMBER,ROUTE_NAME,DIRECTION_NAME,STOP_ID,STOP_NAME,AVERAGE_DAILY_BOARDINGS,AVERAGE_DAILY_ALIGHTINGS,STOP_LAT,STOP_LON
0,2024-08-01,WEEKDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,3051,GRAND WB/RIVIERA FS,43.530172,0.000000,33.988257,-118.469454
1,2024-08-01,WEEKDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,2784,MAIN NB/MARKET NS,203.363246,4.113596,33.988651,-118.471372
2,2024-08-01,WEEKDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,-4,MAIN NB/WESTMINSTER FS,34.237547,0.290544,33.990316,-118.472653


In [13]:
raw_big_blue_bus_export = raw_big_blue_bus.groupby(["SERVICE_PERIOD", "SERVICE_DAY", "ROUTE_NUMBER", "ROUTE_NAME", "DIRECTION_NAME",
                                                    "STOP_ID", "STOP_NAME", "STOP_LAT", "STOP_LON"])[["AVERAGE_DAILY_BOARDINGS", "AVERAGE_DAILY_ALIGHTINGS"]].sum().reset_index()

raw_big_blue_bus_export["start_date"] = raw_big_blue_bus_export["SERVICE_PERIOD"]
raw_big_blue_bus_export["end_date"] = raw_big_blue_bus_export["SERVICE_PERIOD"] + pd.offsets.MonthEnd(4)

raw_big_blue_bus_export.to_excel("transit_agency_ridership_preprocessed_datasets/big_blue_bus_ridership.xlsx")
raw_big_blue_bus_export.head(3)

,SERVICE_PERIOD,SERVICE_DAY,ROUTE_NUMBER,ROUTE_NAME,DIRECTION_NAME,STOP_ID,STOP_NAME,STOP_LAT,STOP_LON,AVERAGE_DAILY_BOARDINGS,AVERAGE_DAILY_ALIGHTINGS,start_date,end_date
0,2024-08-01,SATURDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,-4,MAIN NB/WESTMINSTER FS,33.990316,-118.472653,28.909881,0.111111,2024-08-01,2024-11-30
1,2024-08-01,SATURDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,1000,4TH NB/COLORADO FS (Downtown SM Station),34.014235,-118.492457,116.886196,187.964533,2024-08-01,2024-11-30
2,2024-08-01,SATURDAY,1,Main St & Santa Monica Blvd/UCLA,EASTBOUND,1054,4TH NB/PICO FS,34.010034,-118.487580,26.176476,41.298676,2024-08-01,2024-11-30


#### Caltrain

- Unmerge first column "Month, Year of Date", and rename columns.
- Add start and end date for each period.

In [14]:
raw_caltrain = pd.read_excel("transit_agency_ridership_raw_datasets/Caltrain/Caltrain Average Ridership Estimates - Origin Station Detail (as of August 2025).xlsx")
raw_caltrain = raw_caltrain.reset_index(drop=True)
raw_caltrain["Month, Year of Date"] = raw_caltrain["Month, Year of Date"].ffill()
raw_caltrain = pd.melt(raw_caltrain, id_vars=["Month, Year of Date", "Origin Station", "Caltrain Ridership"])
raw_caltrain = raw_caltrain.rename(columns={"variable": "Date Type",
                                            "value": "Average Ridership",
                                            "Month, Year of Date": "Month"})
raw_caltrain["Date Type"] = raw_caltrain["Date Type"].replace({
                                                                "Average Weekday Ridership": "Weekday",
                                                                "Average Saturday Ridership": "Saturday",
                                                                "Average Sunday Ridership": "Sunday",
                                                                "Average Holiday Ridership": "Holiday"
                                                                })


In [15]:
raw_caltrain["start_date"] = pd.to_datetime(raw_caltrain["Month"], format="%B %Y")
raw_caltrain["end_date"] = raw_caltrain["start_date"] + pd.offsets.MonthEnd(1)

In [16]:
raw_caltrain.to_excel("transit_agency_ridership_preprocessed_datasets/caltrain_ridership.xlsx")
raw_caltrain.head(3)

,Month,Origin Station,Caltrain Ridership,Date Type,Average Ridership,start_date,end_date
0,July 2025,22nd Street,39836.932826,Weekday,1525.276001,2025-07-01,2025-07-31
1,July 2025,Bayshore,6839.629852,Weekday,241.102249,2025-07-01,2025-07-31
2,July 2025,Belmont,20026.449968,Weekday,755.198175,2025-07-01,2025-07-31


#### Fresno Area Express (City of Fresno)

Add day type, start and end date column to fit in staging table schema.

In [17]:
raw_fresno = pd.read_excel("transit_agency_ridership_raw_datasets/City of Fresno/Daily Stop Level Data 9.1.24 - 8.31.25.xlsx")
raw_fresno.head(3)

,Date,StopID,StopLabel,ProjectedBoarding,ProjectedAlighting
0,2024-09-01,5,NE BRAWLEY - SHIELDS,44.691729,0.000000
1,2024-09-01,5,NE BRAWLEY - SHIELDS,0.000000,29.748092
2,2024-09-01,6,SE SHAW - BRAWLEY,7.000000,0.000000


In [18]:
raw_fresno_export = raw_fresno.groupby(by=["Date", "StopID", "StopLabel"])[["ProjectedBoarding", "ProjectedAlighting"]].sum().reset_index()
raw_fresno_export.head(3)

,Date,StopID,StopLabel,ProjectedBoarding,ProjectedAlighting
0,2024-09-01,5,NE BRAWLEY - SHIELDS,44.691729,29.748092
1,2024-09-01,6,SE SHAW - BRAWLEY,7.000000,0.000000
2,2024-09-01,7,SW SHAW - WEST,20.000000,20.000000


In [19]:
raw_fresno_export["start_date"] = raw_fresno_export["Date"]
raw_fresno_export["end_date"] = raw_fresno_export["Date"]
raw_fresno_export["day_type"] = raw_fresno_export["Date"].apply(get_day_type)

raw_fresno_export.to_excel("transit_agency_ridership_preprocessed_datasets/fresno_area_express_ridership.xlsx")

#### Culver City

1. Skip first row (empty) in csv
2. Aggregate numbers of each time period of day to get daily level ridership.

**Note: Stop sequence can be different for one stop.**

In [43]:
raw_culver_city = pd.read_csv("transit_agency_ridership_raw_datasets/Culver City/Ridership by Time Period, Route, and Stop_7_14_25-8_25_25.csv", skiprows=[1])
raw_culver_city_export = raw_culver_city.groupby(['Day Of Week', 'Route', 'Direction', 
                                           'Stop ID', 'Stop Name'])[['Time Period AVG On', 'Time Period AVG Off']].sum() \
                                .reset_index().rename(columns={'Time Period AVG On': 'AVG On',
                                                              'Time Period AVG Off': 'AVG Off'})
raw_culver_city_export['start_date'] = pd.to_datetime('2025-07-14')
raw_culver_city_export['end_date'] = pd.to_datetime('2025-08-25')

raw_culver_city_export["route_id"] = raw_culver_city_export["Route"].str.extract(r"^\s*([A-Za-z0-9 ]+?)\s*-\s*")

day_type_map = {"1-Weekday": "Weekday",
                "2-Saturday": "Saturday",
                "3-Sunday": "Sunday"}
raw_culver_city_export["day_type"] = raw_culver_city_export["Day Of Week"].map(day_type_map)
raw_culver_city_export.to_excel("transit_agency_ridership_preprocessed_datasets/culver_citybus_ridership.xlsx")
raw_culver_city_export.head()

,Day Of Week,Route,Direction,Stop ID,Stop Name,AVG On,AVG Off,start_date,end_date,route_id,day_type
0,1-Weekday,1-Washington Boulevard,Inbound,101,WindwardAve/MainSt,111.2,0.2,2025-07-14,2025-08-25,1,Weekday
1,1-Weekday,1-Washington Boulevard,Inbound,102,Pacific Ave/N Venice Blvd,31.7,1.9,2025-07-14,2025-08-25,1,Weekday
2,1-Weekday,1-Washington Boulevard,Inbound,103,Washington Blvd/Pacific Ave,84.2,9.7,2025-07-14,2025-08-25,1,Weekday
3,1-Weekday,1-Washington Boulevard,Inbound,104,Washington Blvd/Via Dolce,39.4,3.9,2025-07-14,2025-08-25,1,Weekday
4,1-Weekday,1-Washington Boulevard,Inbound,105,Washington Blvd/Via Marina,42.4,4.3,2025-07-14,2025-08-25,1,Weekday


In [45]:
raw_culver_city_export["route_id"].unique()

array(['1', '1c1', '2', '3', '4', '6', '7', 'Rapid 6'], dtype=object)

#### Gold Coast Transit

1. Import one sheet (May 2025 data) for now, which contains most comprehensive columns/info.
2. Add headers/column names.

In [66]:
# specify headers (inferred from other sheets from the same excel)
headers = ['day_of_week', 'route', 'direction', 'stop_id', 'unknown', 'stop_name', 'total_on', 'total_off', 
           'total_activity', 'cumulative_load', 'lat', 'lon']

raw_gct = pd.read_excel("transit_agency_ridership_raw_datasets/Gold Coast Transit/Longitudinal Stop Ridership Data.xls",
                        sheet_name='May_2025_Stop_Ridership', header=None, names=headers)

raw_gct_export = raw_gct.copy()
raw_gct_export['start_date'] = pd.to_datetime('2025-05-01')
raw_gct_export['end_date'] = pd.to_datetime('2025-05-31')

raw_gct_export.to_excel("transit_agency_ridership_preprocessed_datasets/gold_coast_transit_ridership.xlsx")
raw_gct_export.head(3)

,day_of_week,route,direction,stop_id,unknown,stop_name,total_on,total_off,total_activity,cumulative_load,lat,lon,start_date,end_date
0,Weekday,1A,North,4THBST2,19,4th & B St,2,61,62,97,34.198975,-119.179621,2025-05-01,2025-05-31
1,Weekday,1A,South,4THBST1,1,4th & B St,70,2,72,188,34.199066,-119.179574,2025-05-01,2025-05-31
2,Weekday,1B,North,4THBST2,21,4th & B St,1,56,57,115,34.198975,-119.179621,2025-05-01,2025-05-31


#### Golden Gate Park Shuttle

1. Reshape the data from a wide format, where each stop is a separate column, into a long format where each row represents the ridership for a specific stop on a specific day.
2. Filter out Stop = "Total"

In [32]:
raw_ggp = pd.read_excel("transit_agency_ridership_raw_datasets/Golden Gate Park Shuttle/GGPshuttle_RidershipFY25.xlsx", header=None)
raw_ggp.head(3)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,NaN,NaN,NaN,Blue Heron Boathouse,Rose Garden - EB,10th Ave/ De Young EB,De Young Museum,Music Concourse,Academy of Sciences,...,JFK Gateway EB,Haight/Stanyan,JFK Gateway WB,Tennis Center/ Dalia Dell WB,Conservatory of Flowers WB,8th Ave WB,10th Ave/ De Young WB,Rose Garden WB,Transverse,Total
1,Date,Month,Day Type,Day,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-01 00:00:00,7,Weekday,Monday,45,19,0,5,0,22,...,10,10,4,14,11,5,9,0,6,179


In [33]:
# extract firs four column names from the second row (index=1)
first_four_cols = raw_ggp.iloc[1, 0:4].tolist()
first_four_cols

['Date', 'Month', 'Day Type', 'Day']

In [34]:
# extract stop names from the firs row starting from 5th row
stop_names = raw_ggp.iloc[0, 4:].tolist()
stop_names

['Blue Heron Boathouse',
 'Rose Garden - EB',
 '10th Ave/ De Young EB',
 'De Young Museum',
 'Music Concourse',
 'Academy of Sciences',
 '8th Ave EB',
 'Conservatory of Flowers EB',
 'Tennis Center/ Dalia Dell EB',
 'JFK Gateway EB',
 'Haight/Stanyan',
 'JFK Gateway WB',
 'Tennis Center/ Dalia Dell WB',
 'Conservatory of Flowers WB',
 '8th Ave WB',
 '10th Ave/ De Young WB',
 'Rose Garden WB',
 'Transverse',
 'Total']

In [41]:
# combine to create a full list of columns
columns = first_four_cols + stop_names

# extract data starting from 3rd row
data_ggp = raw_ggp.iloc[2:, :].copy()
data_ggp.columns = columns

# convert from wide to long format
raw_ggp_export = data_ggp.melt(id_vars=first_four_cols, var_name="Stop", value_name="Ridership")
raw_ggp_export["start_date"] = pd.to_datetime(raw_ggp_export["Date"])
raw_ggp_export["end_date"] = pd.to_datetime(raw_ggp_export["Date"])
raw_ggp_export["direction"] = raw_ggp_export["Stop"].str.extract(r"\b([EWSN]\s*B)\b",flags=re.IGNORECASE)

raw_ggp_export = raw_ggp_export[raw_ggp_export["Stop"] != "Total"]
raw_ggp_export = raw_ggp_export[~raw_ggp_export["Date"].isna()]

raw_ggp_export.to_excel("transit_agency_ridership_preprocessed_datasets/golden_gate_park_shuttle_ridership.xlsx")

raw_ggp_export.head(3)

,Date,Month,Day Type,Day,Stop,Ridership,start_date,end_date,direction
0,2024-07-01,7,Weekday,Monday,Blue Heron Boathouse,45,2024-07-01,2024-07-01,NaN
1,2024-07-02,7,Weekday,Tuesday,Blue Heron Boathouse,59,2024-07-02,2024-07-02,NaN
2,2024-07-03,7,Weekday,Wednesday,Blue Heron Boathouse,74,2024-07-03,2024-07-03,NaN


In [42]:
raw_ggp_export[raw_ggp_export["direction"].notna()]

,Date,Month,Day Type,Day,Stop,Ridership,start_date,end_date,direction
367,2024-07-01,7,Weekday,Monday,Rose Garden - EB,19,2024-07-01,2024-07-01,EB
368,2024-07-02,7,Weekday,Tuesday,Rose Garden - EB,12,2024-07-02,2024-07-02,EB
369,2024-07-03,7,Weekday,Wednesday,Rose Garden - EB,5,2024-07-03,2024-07-03,EB
370,2024-07-04,7,Weekday,Thursday,Rose Garden - EB,26,2024-07-04,2024-07-04,EB
371,2024-07-05,7,Weekday,Friday,Rose Garden - EB,7,2024-07-05,2024-07-05,EB
...,...,...,...,...,...,...,...,...,...
6232,2025-06-26,6,Weekday,Thursday,Rose Garden WB,38,2025-06-26,2025-06-26,WB
6233,2025-06-27,6,Weekday,Friday,Rose Garden WB,36,2025-06-27,2025-06-27,WB
6234,2025-06-28,6,Weekend,Saturday,Rose Garden WB,7,2025-06-28,2025-06-28,WB
6235,2025-06-29,6,Weekend,Sunday,Rose Garden WB,11,2025-06-29,2025-06-29,WB


#### Long Beach Transit

Aggregate to daily-stop level. Raw data is trip-stop level avg for weekday/weekend.

In [27]:
raw_long_beach = pd.read_excel("transit_agency_ridership_raw_datasets/Long Beach Transit/FY25_LBT_StopLevelRidership_CalITP.xlsx")
raw_long_beach.head(3)

,DayType,Route,Direction,PatternTripID,StopRank,StopID,StopName,ScheduledTime,Boardings,Alightings,Load
0,Weekday,1,Inbound,10000683,0,2030,Victoria & Tamcliff NW,6:41,2.259647,NaN,2.259647
1,Weekday,1,Inbound,10000683,1,2028,Victoria & Rainsbury NW,6:41,1.129823,0.0,3.389470
2,Weekday,1,Inbound,10000683,2,2026,Avalon & Victoria SW,6:42,2.259647,0.0,5.649117


In [28]:
raw_long_beach_export = raw_long_beach.groupby(by=['DayType', 'Route', 'Direction', 'StopID', 'StopName'])[['Boardings', 'Alightings']].sum() \
                                      .reset_index()
fiscal_year = 2025
start_month = 7
start_date, end_date = get_fiscal_year_range(fiscal_year, start_month)
raw_long_beach_export['start_date'] = pd.to_datetime(start_date)
raw_long_beach_export['end_date'] = pd.to_datetime(end_date)
raw_long_beach_export.to_excel("transit_agency_ridership_preprocessed_datasets/long_beach_transit_ridership.xlsx")

raw_long_beach_export.head(3)

,DayType,Route,Direction,StopID,StopName,Boardings,Alightings,start_date,end_date
0,Saturday,1,Inbound,2002,2660 Del Amo Blvd S,0.000000,0.000000,2024-07-01,2025-06-30
1,Saturday,1,Inbound,2004,Del Amo & Fordyce SW,5.977199,2.219394,2024-07-01,2025-06-30
2,Saturday,1,Inbound,2006,Del Amo & Wilmington SW,1.890421,2.872157,2024-07-01,2025-06-30


#### OmniTrans

1. Average to daily. Raw data is total ridership over each fiscal year.
2. Add date according to fiscal year, i.e., first day of the fiscal year.

**Note: For each Stop Name, there can be multiple rows. Need to first sum up for stop name then divided by 365 to get day avg.**

In [29]:
raw_omni = pd.read_excel("transit_agency_ridership_raw_datasets/OmniTrans/Ridership Data_DOT request.xlsx")
raw_omni.head(3)

,FiscalYear,Route,Stop Name,Total Board,Total Alight
0,2025,14,San Bernardino Transit Center,84053,75783
1,2024,61,Fontana Metrolink Transit Center,81335,79081
2,2024,215,San Bernardino Transit Center,80230,76159


In [30]:
# sum up totals for each stop name
raw_omni_export = raw_omni.groupby(["FiscalYear", "Route", "Stop Name"])[["Total Board", "Total Alight"]].sum().reset_index()

# avg total ridership of fiscal year
raw_omni_export['avg_boardings'] = raw_omni_export['Total Board']/365
raw_omni_export['avg_alightings'] = raw_omni_export['Total Alight']/365

# specify start month of fiscal year
start_month = 7
raw_omni_export[["start_date", "end_date"]] = raw_omni_export["FiscalYear"].apply(lambda x: pd.Series(get_fiscal_year_range(x, start_month)))
raw_omni_export["start_date"] = pd.to_datetime(raw_omni_export["start_date"])
raw_omni_export["end_date"] = pd.to_datetime(raw_omni_export["end_date"])
# raw_omni_export['Date'] = pd.to_datetime(raw_omni_export['FiscalYear']-1, format='%Y') + pd.offsets.MonthBegin(start_month-1)

raw_omni_export["day_type"] = "all"

raw_omni_export.to_excel("transit_agency_ridership_preprocessed_datasets/omni_trans_ridership.xlsx")

raw_omni_export.head(3)

,FiscalYear,Route,Stop Name,Total Board,Total Alight,avg_boardings,avg_alightings,start_date,end_date,day_type
0,2024,1,2ND @ F ST,939,226,2.572603,0.619178,2023-07-01,2024-06-30,all
1,2024,1,2ND @ G ST,515,2811,1.410959,7.701370,2023-07-01,2024-06-30,all
2,2024,1,2ND @ J ST,1005,458,2.753425,1.254795,2023-07-01,2024-06-30,all


#### Orange County Transportation Authority

- Aggregate trip-stop/event level (door open/close) ridership to stop level ridership.
- Looks like raw stop name has stop id as prefix. Extract stop id from stop name.

In [47]:
raw_octa = pd.read_excel("transit_agency_ridership_raw_datasets/Orange County Transportation Authority/20250204_StopLevelBoarding.xlsx", skiprows=[1])
raw_octa.head(3)

,Cal Year,Month,Trans Date,Day of Week,Route,Direction,Trip ID,Stop City,Stop Name,APC Open Time,APC Close Time,APC Boarding,APC Alighting
0,2025,Feb,2025-02-04,Tuesday,60-Long Beach - Tustin,E,11669472,GARDEN GROVE,2528-WESTMINSTER-CLINTON,00:00:05,00:00:13,0,1
1,2025,Feb,2025-02-04,Tuesday,57-Brea - Newport Beach,N,11748034,SANTA ANA,8606-BRISTOL-WASHINGTON,00:00:09,00:00:28,0,1
2,2025,Feb,2025-02-04,Tuesday,42-Orange - Seal Beach,W,11794340,BUENA PARK,863-LINCOLN-KNOTT,00:00:13,00:00:38,0,1


In [48]:
raw_octa_export = raw_octa.groupby(by=['Cal Year', 'Month', 'Trans Date', 'Day of Week', 'Route', 'Direction', 'Stop Name'])[['APC Boarding', 'APC Alighting']].sum().reset_index()

raw_octa_export["stop_id"] = raw_octa_export["Stop Name"].str.extract(r"^\s*(\d+)\s*[--]\s*")
raw_octa_export["route_id"] = raw_octa_export["Route"].str.extract(r"^\s*(\d+)\s*[--]\s*")
raw_octa_export["Trans Date"] = pd.to_datetime(raw_octa_export["Trans Date"], errors="coerce")
raw_octa_export["day_type"] = raw_octa_export["Trans Date"].apply(get_day_type)
raw_octa_export["start_date"] = pd.to_datetime(raw_octa_export["Trans Date"])
raw_octa_export["end_date"] = pd.to_datetime(raw_octa_export["Trans Date"])
raw_octa_export.to_excel("transit_agency_ridership_preprocessed_datasets/octa_ridership.xlsx")

raw_octa_export.head()

,Cal Year,Month,Trans Date,Day of Week,Route,Direction,Stop Name,APC Boarding,APC Alighting,stop_id,route_id,day_type,start_date,end_date
0,2025,Feb,2025-02-04,Tuesday,1-Long Beach - San Clemente,N,1501-PACIFIC COAST-DEL OBISPO,36,30,1501,1,weekday,2025-02-04,2025-02-04
1,2025,Feb,2025-02-04,Tuesday,1-Long Beach - San Clemente,N,1503-PACIFIC COAST-ALCAZAR,3,3,1503,1,weekday,2025-02-04,2025-02-04
2,2025,Feb,2025-02-04,Tuesday,1-Long Beach - San Clemente,N,1506-PACIFIC COAST-AMBER LANTERN,6,7,1506,1,weekday,2025-02-04,2025-02-04
3,2025,Feb,2025-02-04,Tuesday,1-Long Beach - San Clemente,N,1508-PACIFIC COAST-SELVA,3,6,1508,1,weekday,2025-02-04,2025-02-04
4,2025,Feb,2025-02-04,Tuesday,1-Long Beach - San Clemente,N,1509-PACIFIC COAST-NIGUEL,13,6,1509,1,weekday,2025-02-04,2025-02-04


#### Santa Cruz Metro

1. Combine four files to one dataset
2. Extract date information from file names
3. Format stop name and agg/sum ridership for the same stop. E.g., "Barack Obama Blvd + W San Carlos" and "Barack Obama Blvd + W San Carlos [0901]".

In [10]:
def parse_date(date_str):
    date_str = date_str.replace("Sept", "Sep")
    for fmt in ["%b%d,%Y", "%B%d,%Y"]:
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d")
        except ValueError:
            continue
    return None

In [11]:
def extract_dates_sct(filename):
    """extract date range from file name"""
    
    # remove spaces to make the structure fixed
    clean_name = filename.replace(" ", "").replace("_", "-")
    
    match = re.search(r"(FY\d{2})RidershipbyStop-(.*)\.xlsx", clean_name, re.IGNORECASE)
    if not match:
        return None, None, None
    
    prefix = match.group(1).upper() # e.g., FY25RidershipbyStop
    date_part = match.group(2) # e.g. Dec19,2024-March12,2025
    
    if "-" in date_part:
        start_str, end_str = date_part.split("-", 1)
    else:
        return prefix, None, None
    
    start_date, end_date = parse_date(start_str), parse_date(end_str)
    
    return prefix, start_date, end_date

In [12]:
def flatten_columns_scm(columns):
    """Flatten a multi-index of two header rows into single column names."""
    
    new_cols = []
    for top, bottom in columns:
        top = str(top).strip() if pd.notna(top) else ""
        bottom = str(bottom).strip() if pd.notna(bottom) else ""
        
        if bottom.startswith("Unnamed"):
            bottom = ""
        if bottom and bottom != top:
            name = f"{top} {bottom}".strip()
        else:
            name = top
            
        new_cols.append(name)
    return new_cols

In [15]:
raw_scm_export = []

for file_path in glob("transit_agency_ridership_raw_datasets/Santa Cruz Metro/*.xlsx"):
    file_name = file_path.split("/")[-1]
    
    prefix, start_date, end_date = extract_dates_sct(file_name)
    
    raw_scm = pd.read_excel(file_path, header=[0,1])
    raw_scm.columns = flatten_columns_scm(raw_scm.columns)
    raw_scm["start_date"] = pd.to_datetime(start_date)
    raw_scm["end_date"] = pd.to_datetime(end_date)
    raw_scm["filename"] = file_name
    raw_scm["Stop Name"] = raw_scm["Stop Name"].str.replace(r"\s*\[.*$", "", regex=True).str.strip()
    raw_scm = raw_scm.groupby(by=["Stop Name", "Stop ID", "start_date", "end_date", "filename"], as_index=False).agg(Boardings=("Boardings", "sum"),
                                                                                                                                 Alightings=("Alightings", "sum")
    )
    # raw_scm.rename(columns={"Load": "Load Avg",
    #                         "Unnamed: 5": "Load Max"}, inplace=True)
    raw_scm_export.append(raw_scm)

raw_scm_export = pd.concat(raw_scm_export)
raw_scm_export["day_type"] = "all"
raw_scm_export.to_excel("transit_agency_ridership_preprocessed_datasets/santa_cruz_metro_ridership.xlsx")
raw_scm_export.head()

,Stop Name,Stop ID,start_date,end_date,filename,Boardings,Alightings,day_type
0,17th Ave + Brommer,2170,2024-09-12,2024-12-18,"FY25 Ridership by Stop_Sept 12, 2024-Dec 18,20...",84.0,234.0,all
1,17th Ave + Felt,2171,2024-09-12,2024-12-18,"FY25 Ridership by Stop_Sept 12, 2024-Dec 18,20...",301.0,512.0,all
2,17th Ave + Felt,2175,2024-09-12,2024-12-18,"FY25 Ridership by Stop_Sept 12, 2024-Dec 18,20...",734.0,265.0,all
3,17th Ave + Kinsley,2178,2024-09-12,2024-12-18,"FY25 Ridership by Stop_Sept 12, 2024-Dec 18,20...",274.0,162.0,all
4,17th Ave + Matthews Ln,2173,2024-09-12,2024-12-18,"FY25 Ridership by Stop_Sept 12, 2024-Dec 18,20...",390.0,349.0,all


#### SunLine Transit

Format the data.

In [37]:
raw_sunline = pd.read_excel("transit_agency_ridership_raw_datasets/SunLine Transit/Ridership Data by stop (APC).xlsx", header=[0,1, 2])
raw_sunline.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
             Stop ID          Stop Name           Latitude          Longitude   
0                  0      (X) Undefined                NaN                NaN   
1                  1        Palm@2Bunch          33.947457        -116.501166   
2                  2      Palm@Hacienda          33.953131        -116.501175   
3                  3      Palm@Estrella          33.956542        -116.501183   
4                  4      Ramon@Farrell          33.816100        -116.518680   

  Unnamed: 4_level_0                         FY 2023                          \
  Unnamed: 4_level_1 (July 1, 2022 to June 30, 2023)                           
   Route Serves Stop                      APC Boards APC Alights Avg. Boards   
0                NaN                            5916        7002       16.21   
1                2,5                            8568       11722       23.47   
2                2,5                           10607       17462       29.06   
3                2,5                            1941        4016        5.32   
4         2400401403                            5625        3010       15.41   

                                       FY 2024  ...                           \
               (July 1, 2023 to June 30, 2024)  ...                            
  Avg. Alights                      APC Boards  ... Avg. Boards Avg. Alights   
0        19.18                            5668  ...       15.53        16.19   
1        32.12                            9453  ...        25.9           29   
2        47.84                           11411  ...       31.26        49.95   
3           11                            1737  ...        4.76        10.53   
4         8.25                            5739  ...       15.72          7.3   

                          FY 2025                                       \
  (July 1, 2024 to June 30, 2025)                                        
                       APC Boards APC Alights Avg. Boards Avg. Alights   
0                            5115        4914       14.01        13.46   
1                            7685        9027       21.05        24.73   
2                            9089       16094        24.9        44.09   
3                            1516        3659        4.15        10.02   
4                            2311        1054        6.33         2.89   

                      Spring 2025                                       
  (March 1, 2025 to May 31, 2025)                                       
                       APC Boards APC Alights Avg. Boards Avg. Alights  
0                           949.0       971.0       10.32        10.55  
1                          1434.0      1866.0       15.59        20.28  
2                          1919.0      3333.0       20.86        36.23  
3                           268.0       624.0        2.91         6.78  
4                           679.0       239.0        7.38         2.60  

[5 rows x 21 columns]

In [38]:
static_cols = ["Stop ID", "Stop Name", "Latitude", "Longitude", "Route Serves Stop"]
raw_sunline.columns = [static_cols[i] if i < len(static_cols) else raw_sunline.columns[i] for i in range(len(raw_sunline.columns))]

In [39]:
ridership_cols = ["APC Boards", "APC Alights", "Avg. Boards", "Avg. Alights"]
ridership_cols_raw = [col for col in raw_sunline.columns if col not in static_cols]
ridership_cols_raw

[('FY 2023', '(July 1, 2022 to June 30, 2023)', 'APC Boards'),
 ('FY 2023', '(July 1, 2022 to June 30, 2023)', 'APC Alights'),
 ('FY 2023', '(July 1, 2022 to June 30, 2023)', 'Avg. Boards'),
 ('FY 2023', '(July 1, 2022 to June 30, 2023)', 'Avg. Alights'),
 ('FY 2024', '(July 1, 2023 to June 30, 2024)', 'APC Boards'),
 ('FY 2024', '(July 1, 2023 to June 30, 2024)', 'APC Alights'),
 ('FY 2024', '(July 1, 2023 to June 30, 2024)', 'Avg. Boards'),
 ('FY 2024', '(July 1, 2023 to June 30, 2024)', 'Avg. Alights'),
 ('FY 2025', '(July 1, 2024 to June 30, 2025)', 'APC Boards'),
 ('FY 2025', '(July 1, 2024 to June 30, 2025)', 'APC Alights'),
 ('FY 2025', '(July 1, 2024 to June 30, 2025)', 'Avg. Boards'),
 ('FY 2025', '(July 1, 2024 to June 30, 2025)', 'Avg. Alights'),
 ('Spring 2025', '(March 1, 2025 to May 31, 2025)', 'APC Boards'),
 ('Spring 2025', '(March 1, 2025 to May 31, 2025)', 'APC Alights'),
 ('Spring 2025', '(March 1, 2025 to May 31, 2025)', 'Avg. Boards'),
 ('Spring 2025', '(March 1, 2

In [40]:
meta = []
for top, mid, bottom in ridership_cols_raw:
    meta.append({
                "col": (top, mid, bottom),
                "period": str(top).strip(),
                "date_range": str(mid).strip(),
                "metric": str(bottom).strip()
    })
df_meta = pd.DataFrame(meta)

# extract start and end date
df_meta[['start_date_str', 'end_date_str']] = df_meta['date_range'].str.extract(r'(\w+\s*\d{1,2},\s*\d{4})\s*to\s*(\w+\s*\d{1,2},\s*\d{4})')
df_meta['start_date'] = pd.to_datetime(df_meta['start_date_str'])
df_meta['end_date'] = pd.to_datetime(df_meta['end_date_str'])
df_meta['agg_basis'] = df_meta['period'].apply(lambda x: 'fiscal year' if isinstance(x, str) and 'FY' in x else 'season')
df_meta

,col,period,date_range,metric,start_date_str,end_date_str,start_date,end_date,agg_basis
0,"(FY 2023, (July 1, 2022 to June 30, 2023), APC...",FY 2023,"(July 1, 2022 to June 30, 2023)",APC Boards,"July 1, 2022","June 30, 2023",2022-07-01,2023-06-30,fiscal year
1,"(FY 2023, (July 1, 2022 to June 30, 2023), APC...",FY 2023,"(July 1, 2022 to June 30, 2023)",APC Alights,"July 1, 2022","June 30, 2023",2022-07-01,2023-06-30,fiscal year
2,"(FY 2023, (July 1, 2022 to June 30, 2023), Avg...",FY 2023,"(July 1, 2022 to June 30, 2023)",Avg. Boards,"July 1, 2022","June 30, 2023",2022-07-01,2023-06-30,fiscal year
3,"(FY 2023, (July 1, 2022 to June 30, 2023), Avg...",FY 2023,"(July 1, 2022 to June 30, 2023)",Avg. Alights,"July 1, 2022","June 30, 2023",2022-07-01,2023-06-30,fiscal year
4,"(FY 2024, (July 1, 2023 to June 30, 2024), APC...",FY 2024,"(July 1, 2023 to June 30, 2024)",APC Boards,"July 1, 2023","June 30, 2024",2023-07-01,2024-06-30,fiscal year
5,"(FY 2024, (July 1, 2023 to June 30, 2024), APC...",FY 2024,"(July 1, 2023 to June 30, 2024)",APC Alights,"July 1, 2023","June 30, 2024",2023-07-01,2024-06-30,fiscal year
6,"(FY 2024, (July 1, 2023 to June 30, 2024), Avg...",FY 2024,"(July 1, 2023 to June 30, 2024)",Avg. Boards,"July 1, 2023","June 30, 2024",2023-07-01,2024-06-30,fiscal year
7,"(FY 2024, (July 1, 2023 to June 30, 2024), Avg...",FY 2024,"(July 1, 2023 to June 30, 2024)",Avg. Alights,"July 1, 2023","June 30, 2024",2023-07-01,2024-06-30,fiscal year
8,"(FY 2025, (July 1, 2024 to June 30, 2025), APC...",FY 2025,"(July 1, 2024 to June 30, 2025)",APC Boards,"July 1, 2024","June 30, 2025",2024-07-01,2025-06-30,fiscal year
9,"(FY 2025, (July 1, 2024 to June 30, 2025), APC...",FY 2025,"(July 1, 2024 to June 30, 2025)",APC Alights,"July 1, 2024","June 30, 2025",2024-07-01,2025-06-30,fiscal year


In [41]:
# loop over ridership metric columns and build long-format rows
long_frames = []

for _, row in df_meta.iterrows():
    col = row['col']
    metric = row['metric']
    # print(col)
    # print(metric)
    
    t_df_static = raw_sunline[static_cols]
    t_df_static['metric'] = metric
    t_df_static['value'] = raw_sunline[col]
    
    t_df_static['period'] = row['period']
    t_df_static['start_date'] = row['start_date']
    t_df_static['end_date'] = row['end_date']
    t_df_static['agg_basis'] = row['agg_basis']
    
    # print('\n')
    long_frames.append(t_df_static)
    
t_df_long = pd.concat(long_frames, ignore_index=True)
t_df_long.head(3)

,Stop ID,Stop Name,Latitude,Longitude,Route Serves Stop,metric,value,period,start_date,end_date,agg_basis
0,0,(X) Undefined,NaN,NaN,NaN,APC Boards,5916,FY 2023,2022-07-01,2023-06-30,fiscal year
1,1,Palm@2Bunch,33.947457,-116.501166,"2,5",APC Boards,8568,FY 2023,2022-07-01,2023-06-30,fiscal year
2,2,Palm@Hacienda,33.953131,-116.501175,"2,5",APC Boards,10607,FY 2023,2022-07-01,2023-06-30,fiscal year


In [42]:
# pivot ridership metrics into columns
raw_sunline_export = t_df_long.pivot_table(
                                            index=["Stop ID", "Stop Name", "Latitude", "Longitude", "Route Serves Stop", "period", "start_date", "end_date", "agg_basis"],
                                            columns='metric',
                                            values='value',
                                            aggfunc='first' # when multiple rows have same index+column values, take the first non-null row
                                            ).reset_index()
raw_sunline_export.to_excel("transit_agency_ridership_preprocessed_datasets/sunline_transit_ridership.xlsx")
raw_sunline_export.head(3)

metric,Stop ID,Stop Name,Latitude,Longitude,Route Serves Stop,period,start_date,end_date,agg_basis,APC Alights,APC Boards,Avg. Alights,Avg. Boards
0,1,Palm@2Bunch,33.947457,-116.501166,"2,5",FY 2023,2022-07-01,2023-06-30,fiscal year,11722,8568,32.12,23.47
1,1,Palm@2Bunch,33.947457,-116.501166,"2,5",FY 2024,2023-07-01,2024-06-30,fiscal year,10585,9453,29,25.9
2,1,Palm@2Bunch,33.947457,-116.501166,"2,5",FY 2025,2024-07-01,2025-06-30,fiscal year,9027,7685,24.73,21.05


#### Golden Gate Transit

Aggregate to stop level. The raw data has trip-stop grain.

In [43]:
raw_ggt = pd.read_csv("transit_agency_ridership_raw_datasets/Golden Gate Transit/RidershipPerStop_September2025.csv")
raw_ggt.head()

,OPERATION_DATE,ROUTE,TRIP_NUMBER,TRIP_NAME,DIRECTION,STOP_NUMBER,STOP_NAME,POINT_ROLE,ARRIVAL_TIME,BOARDINGS,ALIGHTINGS,PASSENGER_LOAD
0,01-SEP-25,101,1801,Route 101 Southbound - San Francisco (101:584),South,40949,Piner Rd & Industrial Dr (40949),ST,03:49:34,0.0,0.0,0.0
1,01-SEP-25,101,1801,Route 101 Southbound - San Francisco (101:584),South,40950,Piner Rd & Range Ave (40950),S,03:50:03,0.0,0.0,0.0
2,01-SEP-25,101,1801,Route 101 Southbound - San Francisco (101:584),South,42126,Santa Rosa Transit Mall (42126),ST,03:55:50,1.0,0.0,1.0
3,01-SEP-25,101,1801,Route 101 Southbound - San Francisco (101:584),South,42121,Rohnert Park Expy & Hwy 101 (42121),S,04:04:00,0.0,0.0,1.0
4,01-SEP-25,101,1801,Route 101 Southbound - San Francisco (101:584),South,40889,Commerce Blvd N & Rohnert Park Expy (40889),ST,04:05:00,1.0,0.0,2.0


In [44]:
raw_ggt[(raw_ggt["POINT_ROLE"] == "T")]['ALIGHTINGS'].sum()

0.0

In [45]:
# filter out virtual time points
t_df_ggt = raw_ggt[raw_ggt["POINT_ROLE"].isin(["ST", "S"])]

raw_ggt_export = raw_ggt.groupby(by=["OPERATION_DATE", "ROUTE", "DIRECTION", "STOP_NUMBER", "STOP_NAME"])[["BOARDINGS", "ALIGHTINGS"]].sum().reset_index()
raw_ggt_export["date"] = pd.to_datetime(raw_ggt_export["OPERATION_DATE"], format="%d-%b-%y")
raw_ggt_export["start_date"] = raw_ggt_export["date"]
raw_ggt_export["end_date"] = raw_ggt_export["date"]
raw_ggt_export["day_type"] = raw_ggt_export["date"].apply(get_day_type)
raw_ggt_export.to_excel("transit_agency_ridership_preprocessed_datasets/golden_gate_transit_ridership.xlsx")
raw_ggt_export.head(3)

,OPERATION_DATE,ROUTE,DIRECTION,STOP_NUMBER,STOP_NAME,BOARDINGS,ALIGHTINGS,date,start_date,end_date,day_type
0,01-SEP-25,101,North,40003,Salesforce Transit Center-Bus Plaza Bay A (40003),36.0,0.0,2025-09-01,2025-09-01,2025-09-01,holiday
1,01-SEP-25,101,North,40024,McAllister St & Polk St (40024),53.0,8.0,2025-09-01,2025-09-01,2025-09-01,holiday
2,01-SEP-25,101,North,40026,Van Ness Ave & Geary Blvd (40026),28.0,3.0,2025-09-01,2025-09-01,2025-09-01,holiday


#### San Diego MTS

Agg to stop level. The raw data is avg ridership for each trip-stop.

**Note: Same stop ID can have more than on Stop Sequence in this dataset. In agg, stop sequence is not included.**

In [46]:
raw_mts_bus = pd.read_csv("transit_agency_ridership_raw_datasets/SDMTS/MTS Ridership(MTS Bus All Days).csv")
raw_mts_trolley = pd.read_csv("transit_agency_ridership_raw_datasets/SDMTS/MTS Ridership(MTS Trolley All Days).csv")
raw_mts = pd.concat([raw_mts_bus, raw_mts_trolley])
raw_mts.head(1)

,Schedule Period,Day Of Week,Route,Route Name,Stop ID,Stop Sequence,Stop Name,Direction ID,Direction Label,Trip ID,Trip Start Time (HHMM),Block,Scheduled_Time,AVG_Arrival_Time,AVG_Depart_Time,Dwell Time (Decimal Mins.),Average On,Average Off,Average Load,Interstop Distance
0,"September 1, 2024 - January 25, 2025",1-Weekday,1,1:Fashion Valley-La Mesa,94048,1,Fashion Valley Transit Center,0,East,18044647,503,900102,5:03:00,4:47:35,5:02:18,14.709277,2.39759,0.0,2.39759,0.0


In [47]:
raw_mts.columns = raw_mts.columns.str.strip()
raw_mts_export = raw_mts.groupby(["Schedule Period", "Day Of Week", "Route", "Route Name", "Stop ID", "Stop Name",
                                          "Direction ID", "Direction Label"])[["Average On", "Average Off"]].sum().reset_index()
day_type_map = {"1-Weekday": "Weekday",
                "2-Saturday": "Saturday",
                "3-Sunday": "Sunday"}
raw_mts_export["day_type"] = raw_mts_export["Day Of Week"].map(day_type_map)
raw_mts_export[["start_date_str", "end_date_str"]] = raw_mts_export["Schedule Period"].str.extract(r'([A-Za-z]+\s*\d{1,2},\s*\d{4})\s*-\s*([A-Za-z]+\s*\d{1,2},\s*\d{4})')
raw_mts_export["start_date"] = pd.to_datetime(raw_mts_export["start_date_str"])
raw_mts_export["end_date"] = pd.to_datetime(raw_mts_export["end_date_str"])
raw_mts_export.drop(columns=["start_date_str", "end_date_str"], inplace=True)
raw_mts_export.to_excel("transit_agency_ridership_preprocessed_datasets/sdmts_ridership.xlsx")
raw_mts_export.head(3)

,Schedule Period,Day Of Week,Route,Route Name,Stop ID,Stop Name,Direction ID,Direction Label,Average On,Average Off,day_type,start_date,end_date
0,"September 1, 2024 - January 25, 2025",1-Weekday,1,1:Fashion Valley-La Mesa,10106,University Av & 10th Av,0,East,26.228026,15.629832,Weekday,2024-09-01,2025-01-25
1,"September 1, 2024 - January 25, 2025",1-Weekday,1,1:Fashion Valley-La Mesa,10111,University Av & Vermont St,0,East,59.583915,17.940893,Weekday,2024-09-01,2025-01-25
2,"September 1, 2024 - January 25, 2025",1-Weekday,1,1:Fashion Valley-La Mesa,10114,University Av & Richmond St,0,East,14.963778,14.173884,Weekday,2024-09-01,2025-01-25


#### SacRT

1. Join ridership and stops table
2. Differentiate weekday and weekend ridership

**Note: For the routes that run on weekends, weekends are included in the agg counts. For the routes that don't run on weekends, only weekdays are included in agg conuts. See the indicator columns from Monday to Sunday.**

In [48]:
raw_sacrt_ridership = pd.read_csv("transit_agency_ridership_raw_datasets/SacRT/ridership.txt.csv")
raw_sacrt_ridership.columns = raw_sacrt_ridership.columns.str.lower()

# cast indicator cols to int
dow_cols = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
raw_sacrt_ridership[dow_cols] = raw_sacrt_ridership[dow_cols].apply(pd.to_numeric, errors="coerce").astype("Int64")

raw_sacrt_ridership = raw_sacrt_ridership.loc[:, ~raw_sacrt_ridership.columns.str.contains("^unnamed", case=False)]

raw_sacrt_ridership = raw_sacrt_ridership.groupby(["ridership_start_date", "ridership_end_date", "service_start_time", "service_end_time",
                                                   "monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday", "route_id",
                                                   "direction_id", "stop_id"], dropna=False)[["average_boardings", "average_alightings"]].sum().reset_index()
raw_sacrt_ridership.head(3)

,ridership_start_date,ridership_end_date,service_start_time,service_end_time,monday,tuesday,wednesday,thursday,friday,saturday,sunday,route_id,direction_id,stop_id,average_boardings,average_alightings
0,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,0,033,0,536,3,4
1,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,0,033,0,537,4,3
2,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,0,033,0,5256,23,12


In [49]:
raw_sacrt_stops = pd.read_csv("transit_agency_ridership_raw_datasets/SacRT/stops.txt.csv")
raw_sacrt_stops = raw_sacrt_stops.dropna(how="all")
raw_sacrt_stops['stop_id'] = raw_sacrt_stops['stop_id'].astype("int64")
raw_sacrt_stops['stop_code'] = raw_sacrt_stops['stop_code'].astype("int64")
raw_sacrt_stops.head(3)

,stop_lat,stop_code,stop_lon,stop_id,stop_name,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,38.625807,103,-121.475353,103,NORTHGATE BLVD & WINTER GARDEN AVE (NB),NaN,NaN,NaN
1,38.525752,105,-121.520596,105,I-5 FWY & SEAMAS AVE (NB),NaN,NaN,NaN
2,38.574230,107,-121.503227,107,Q ST & 5TH ST (EB),NaN,NaN,NaN


In [50]:
raw_sacrt_routes = pd.read_csv("transit_agency_ridership_raw_datasets/SacRT/routes.txt.csv")
raw_sacrt_routes = raw_sacrt_routes.dropna(how="all")
raw_sacrt_routes['route_type'] = raw_sacrt_routes['route_type'].astype("int64")
raw_sacrt_routes.head(3)

,route_long_name,route_type,route_text_color,route_color,agency_id,route_id,route_short_name
0,GREENBACK,3,FFFFFF,008000,SRTD,001,1
1,NATOMAS/LAND PARK,3,000000,00FF00,SRTD,011,11
2,NATOMAS/ARDEN,3,000000,FFFF00,SRTD,013,13


In [51]:
raw_sacrt = pd.merge(raw_sacrt_ridership, raw_sacrt_stops, how="left", on="stop_id")
raw_sacrt = pd.merge(raw_sacrt, raw_sacrt_routes, how="left", on="route_id")
raw_sacrt = raw_sacrt.loc[:, ~raw_sacrt.columns.str.contains("^unnamed", case=False)]
raw_sacrt.head(3)

,ridership_start_date,ridership_end_date,service_start_time,service_end_time,monday,tuesday,wednesday,thursday,friday,saturday,...,stop_lat,stop_code,stop_lon,stop_name,route_long_name,route_type,route_text_color,route_color,agency_id,route_short_name
0,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,38.595230,536.0,-121.483711,RICHARDS BLVD & DOS RIOS ST (EB),DOS RIOS,3,FFFFFF,800000,SRTD,33
1,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,38.594193,537.0,-121.480525,SUNBEAM AVE & RICHARDS BLVD (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33
2,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,38.590122,5256.0,-121.482307,AHERN ST & NORTH C ST (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33


In [52]:
raw_sacrt_export = raw_sacrt.copy()
raw_sacrt_export["start_date"] = pd.to_datetime(raw_sacrt_export["ridership_start_date"], format="%Y%m%d").dt.strftime("%Y-%m-%d")
raw_sacrt_export["end_date"] = pd.to_datetime(raw_sacrt_export["ridership_end_date"], format="%Y%m%d").dt.strftime("%Y-%m-%d")
raw_sacrt_export["start_date"] = pd.to_datetime(raw_sacrt_export["start_date"])
raw_sacrt_export["end_date"] = pd.to_datetime(raw_sacrt_export["end_date"])

raw_sacrt_export.head(3)

,ridership_start_date,ridership_end_date,service_start_time,service_end_time,monday,tuesday,wednesday,thursday,friday,saturday,...,stop_lon,stop_name,route_long_name,route_type,route_text_color,route_color,agency_id,route_short_name,start_date,end_date
0,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,-121.483711,RICHARDS BLVD & DOS RIOS ST (EB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31
1,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,-121.480525,SUNBEAM AVE & RICHARDS BLVD (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31
2,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,-121.482307,AHERN ST & NORTH C ST (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31


In [53]:
dow_cols = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
unique_pat = raw_sacrt_export[dow_cols].drop_duplicates()
unique_pat

,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,1,1,1,1,1,0,0
1748,1,1,1,1,1,1,1
4046,1,<NA>,1,1,1,1,1


In [54]:
raw_sacrt_export[(raw_sacrt_export["saturday"] == 0) & (raw_sacrt_export["sunday"] == 0)]["route_id"].unique()

array(['033', '10', '102', '103', '105', '106', '109', '113', '124',
       '129', '134', '137', '138', '161', '175', '176', '177', '205',
       '206', '210', '211', '212', '213', '214', '215', '226', '227',
       '228', '246', '247', '248', '252', '255', 'F20'], dtype=object)

In [55]:
raw_sacrt_export[(raw_sacrt_export["saturday"] == 1) & (raw_sacrt_export["sunday"] == 1)]["route_id"].unique()

array(['001', '011', '013', '015', '019', '021', '023', '025', '026',
       '030', '038', '051', '056', '061', '062', '067', '068', '072',
       '075', '078', '081', '082', '084', '086', '087', '088', '093',
       '142'], dtype=object)

In [56]:
raw_sacrt_export.groupby("route_type").size()

route_type
3    4047
dtype: int64

In [57]:
# add day type label

dow_cols = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

def day_type_label(row):
    days = row[dow_cols].tolist()
    # print(days)
    
    # handle missing values as 0 for comparison (currently only one record has missing value in Tuesday columns)
    days = [0 if pd.isna(x) else x for x in days]
    
    weekends = days[5:]
    if weekends == [0, 0]:
        return "weekday"
    elif days == [1,1,1,1,1,1,1]:
        return "daily"
    else:
        return "other"

raw_sacrt_export["day_type"] = raw_sacrt_export.apply(day_type_label, axis=1)

raw_sacrt_export_bus = raw_sacrt_export[raw_sacrt_export["route_type"] == 3]
raw_sacrt_export_bus.to_excel("transit_agency_ridership_preprocessed_datasets/sacrt_bus_ridership.xlsx")

raw_sacrt_export_lr = raw_sacrt_export[raw_sacrt_export["route_type"] == 0]
if len(raw_sacrt_export_lr):
    raw_sacrt_export_lr.to_excel("transit_agency_ridership_preprocessed_datasets/sacrt_lightrail_ridership.xlsx")

raw_sacrt_export.head(3)

,ridership_start_date,ridership_end_date,service_start_time,service_end_time,monday,tuesday,wednesday,thursday,friday,saturday,...,stop_name,route_long_name,route_type,route_text_color,route_color,agency_id,route_short_name,start_date,end_date,day_type
0,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,RICHARDS BLVD & DOS RIOS ST (EB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31,weekday
1,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,SUNBEAM AVE & RICHARDS BLVD (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31,weekday
2,20230901,20231231,00:00:00,24:00:00,1,1,1,1,1,0,...,AHERN ST & NORTH C ST (SB),DOS RIOS,3,FFFFFF,800000,SRTD,33,2023-09-01,2023-12-31,weekday


In [58]:
raw_sacrt_export[raw_sacrt_export["day_type"] == "other"]

,ridership_start_date,ridership_end_date,service_start_time,service_end_time,monday,tuesday,wednesday,thursday,friday,saturday,...,stop_name,route_long_name,route_type,route_text_color,route_color,agency_id,route_short_name,start_date,end_date,day_type
4046,20230901,20231231,00:00:00,24:00:00,1,<NA>,1,1,1,1,...,3RD ST & K ST (NB),FREEPORT,3,FFFFFF,008000,SRTD,62,2023-09-01,2023-12-31,other


#### SamTrans

1. Combine files into one dataset
2. Aggregate trip-stop to stop level ridership.

**Note: The data is for each trip-stop, including door open and close time. The lat and lon at the same stop can be slightly different across trips. The agg in the preprocessing takes maximum of the lat and lon for each stop across trips.**

In [59]:
base_path = "transit_agency_ridership_raw_datasets/SamTrans"
files = glob(os.path.join(base_path, "**", "*.xlsx"), recursive=True)

raw_samtrans = []

for file in files:
    t_df = pd.read_excel(file)
    raw_samtrans.append(t_df)

raw_samtrans = pd.concat(raw_samtrans, ignore_index=True)
raw_samtrans.head(3)

/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

,Route,Run,Schedule,Vehicle,Sequence,APC Date,Stop ID,Stop Name,Door Open,Door Close,...,Ons,Offs,Max Load,Rev Distance,Pos Src,Qual Ind,Num Sat,Lat,Lon,Description
0,0,4014.0,10006760.0,452.0,2.0,2025-08-22,341153.0,El Camino Real & Hillsdale Blvd,2025-08-22 00:45:06,2025-08-22 00:47:50,...,3.0,2.0,1.0,0.000000,1.0,6.0,8.0,37.536121,-122.296707,No Error
1,0,603.0,10015713.0,852.0,2.0,2025-08-21,331059.0,Mission St & 1st St (Northbound),2025-08-21 04:03:57,2025-08-21 04:03:57,...,6.0,0.0,6.0,0.000000,1.0,4.0,5.0,37.790493,-122.397140,No Error
2,10,65.0,101001.0,705.0,2.0,2025-08-21,332021.0,Daly City BART-Bay 4,2025-08-21 07:31:02,2025-08-21 07:31:38,...,7.0,0.0,7.0,0.832734,1.0,6.0,9.0,37.705936,-122.468894,No Error


In [60]:
cols_type = ["Run", "Schedule", "Vehicle", "Sequence", "Stop ID", "Ons", "Offs", "Pos Src", "Qual Ind", "Num Sat"]
raw_samtrans[cols_type] = raw_samtrans[cols_type].astype("Int64")
raw_samtrans["APC Date"] = pd.to_datetime(raw_samtrans["APC Date"])
raw_samtrans.head(3)

,Route,Run,Schedule,Vehicle,Sequence,APC Date,Stop ID,Stop Name,Door Open,Door Close,...,Ons,Offs,Max Load,Rev Distance,Pos Src,Qual Ind,Num Sat,Lat,Lon,Description
0,0,4014,10006760,452,2,2025-08-22,341153,El Camino Real & Hillsdale Blvd,2025-08-22 00:45:06,2025-08-22 00:47:50,...,3,2,1.0,0.000000,1,6,8,37.536121,-122.296707,No Error
1,0,603,10015713,852,2,2025-08-21,331059,Mission St & 1st St (Northbound),2025-08-21 04:03:57,2025-08-21 04:03:57,...,6,0,6.0,0.000000,1,4,5,37.790493,-122.397140,No Error
2,10,65,101001,705,2,2025-08-21,332021,Daly City BART-Bay 4,2025-08-21 07:31:02,2025-08-21 07:31:38,...,7,0,7.0,0.832734,1,6,9,37.705936,-122.468894,No Error


In [61]:
raw_samtrans_export = raw_samtrans.groupby(["Route", "APC Date", "Stop ID", "Stop Name"]).agg({"Ons": "sum",
                                                                                             "Offs": "sum",
                                                                                             "Lat": "max",
                                                                                             "Lon": "max"}).reset_index()
raw_samtrans_export["day_type"] = raw_samtrans_export["APC Date"].apply(get_day_type)
raw_samtrans_export["start_date"] = raw_samtrans_export["APC Date"]
raw_samtrans_export["end_date"] = raw_samtrans_export["APC Date"]
raw_samtrans_export.to_excel("transit_agency_ridership_preprocessed_datasets/samtrans_ridership.xlsx")
raw_samtrans_export.head(3)

,Route,APC Date,Stop ID,Stop Name,Ons,Offs,Lat,Lon,day_type,start_date,end_date
0,0,2025-08-12,334620,South SF BART-Bay 5 Westside Busway,0,0,37.664192,-122.444625,weekday,2025-08-12,2025-08-12
1,0,2025-08-21,331059,Mission St & 1st St (Northbound),6,0,37.790493,-122.397140,weekday,2025-08-21,2025-08-21
2,0,2025-08-22,341153,El Camino Real & Hillsdale Blvd,3,2,37.536121,-122.296707,weekday,2025-08-22,2025-08-22


#### Santa Barbara Metropolitan Transit District

1. Reformat to long format
2. Average from monthly to daily

In [62]:
def process_sheet_sbmtd(sheet_name):
    
    t_df_sbmtd = pd.read_excel("transit_agency_ridership_raw_datasets/SBMTD/SBMTD Stop and Route Level Ridership Nov 1 2024 to Oct 31 2025 - date_fixed.xlsx",
                              sheet_name=sheet_name, header=2)
    # drop "Grand Total" col, and "Total" row
    t_df_sbmtd = t_df_sbmtd.loc[:, ~t_df_sbmtd.columns.str.contains("Grand Total", case=False, na=False)]
    t_df_sbmtd = t_df_sbmtd[~t_df_sbmtd.iloc[:, 0].astype(str).str.strip().str.lower().eq("total")]

    # identify stop id and stop name cols and month cols
    id_cols = t_df_sbmtd.columns[:2].tolist()
    month_cols = t_df_sbmtd.columns[2:].tolist()

    # melt into long format
    t_df_sbmtd_long = t_df_sbmtd.melt(id_vars=id_cols, var_name="month_str", value_name=sheet_name)

    # get start and end date for each month
    t_df_sbmtd_long["start_date"] = t_df_sbmtd_long["month_str"].apply(lambda x: x.replace(day=1) if pd.notnull(x) else None)
    t_df_sbmtd_long["end_date"] = t_df_sbmtd_long["month_str"].apply(lambda x: x.replace(day=1) + pd.tseries.offsets.MonthEnd(1) if pd.notnull(x) else None)
    
    t_df_sbmtd_long.columns = t_df_sbmtd_long.columns.str.strip()
    return t_df_sbmtd_long

In [63]:
sheet_names = ["Ridership by Stop_Boardings", "Ridership by Stop_Alightings", "Ridership by Stop_Total Act."]
t_dfs = [process_sheet_sbmtd(sheet_name) for sheet_name in sheet_names]

# merge all three sheets on stop id, stop name and month str
raw_sbmtd = t_dfs[0]
for t_df in t_dfs[1:]:
    # print(t_df.head())
    raw_sbmtd = pd.merge(raw_sbmtd, t_df, on=["Stop ID", "Stop Name", "month_str", "start_date", "end_date"])

raw_sbmtd["avg_boardings"] = 1.0*raw_sbmtd["Ridership by Stop_Boardings"]/30
raw_sbmtd["avg_alightings"] = 1.0*raw_sbmtd["Ridership by Stop_Alightings"]/30
raw_sbmtd["avg_ridership"] = 1.0*raw_sbmtd["Ridership by Stop_Total Act."]/30

raw_sbmtd["day_type"] = "all"
raw_sbmtd.to_excel("transit_agency_ridership_preprocessed_datasets/sbmtd_ridership.xlsx")

#### Riverside Transit

Aggregate swipe/transaction level data to stop level ridership

**Note:** The original raw datasets are transactions data, which are too large to directly upload to Github. The code in this section preprocess the raw data and aggregate to stop level ridership. Raw datasets are not uploaded to Github repo. Lat and lon for each (stop, route, direction) are the max/min of all records for the corresponding combination.

In [8]:
raw_riverside_sample  = pd.read_csv("transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(SEPTEMBER_1_2025-OCTOBER_31_2025).csv", header=4)
raw_riverside_sample = raw_riverside_sample[raw_riverside_sample["Transaction Type"].isin(["114 - Stored ride card", "115 - Period pass", "119 - Got fare", 
                                                       "129 - Special card", "212 - Mobile Ticket Transaction"])]
raw_riverside_sample["date"] = pd.to_datetime(raw_riverside_sample["Date Time"], errors="coerce").dt.date
raw_riverside_sample["date"] = pd.to_datetime(raw_riverside_sample["date"], errors="coerce")
raw_riverside_sample["Route"] = raw_riverside_sample["Route"].astype(int)
raw_riverside_sample["Stop ID"] = raw_riverside_sample["Stop ID"].astype(int)

raw_riverside_sample.head()

/tmp/ipykernel_261/2014185633.py:1: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_riverside_sample  = pd.read_csv("transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(SEPTEMBER_1_2025-OCTOBER_31_2025).csv", header=4)


,Location,Transaction Type,Product,Date Time,Bus,Driver,Route,Run,Trip,FS,...,Sec Cd,Longitude,Latitude,Imp. Val,Amt Chrg,Rem Val,Stop ID,Stop Abbr,Stop Name,date
0,2,119 - Got fare,GEN/YTH$,09/01/2025 00:00:00,31382.0,102449.0,28,28804.0,863288.0,1.0,...,NaN,-117.108352,33.743296,NaN,0.25,NaN,2776,NaN,NaN,2025-09-01
1,2,119 - Got fare,DUMP,09/01/2025 00:02:00,31382.0,102449.0,28,28804.0,863288.0,1.0,...,NaN,-117.108352,33.743296,NaN,0.01,NaN,2777,NaN,NaN,2025-09-01
2,1,119 - Got fare,DUMP,09/02/2025 03:17:44,31643.0,100953.0,200,200101.0,858901.0,2.0,...,NaN,0.000000,0.000000,NaN,0.01,NaN,999999,NaN,NaN,2025-09-02
3,1,119 - Got fare,DUMP,09/02/2025 03:24:47,32345.0,101462.0,1,14101.0,858947.0,1.0,...,NaN,0.000000,0.000000,NaN,0.25,NaN,1007,NaN,NaN,2025-09-02
4,1,119 - Got fare,SEN/DIS$,09/02/2025 03:29:00,32345.0,101462.0,1,14101.0,858947.0,1.0,...,NaN,-117.564864,33.882060,NaN,0.75,NaN,3426,NaN,NaN,2025-09-02


In [12]:
raw_riverside_sample[(raw_riverside_sample["Stop ID"] > 5000) & (raw_riverside_sample["Stop ID"] != 999999)]

,Location,Transaction Type,Product,Date Time,Bus,Driver,Route,Run,Trip,FS,...,Sec Cd,Longitude,Latitude,Imp. Val,Amt Chrg,Rem Val,Stop ID,Stop Abbr,Stop Name,date
984084,4,119 - Got fare,DUMP,10/25/2025 15:11:46,22011.0,0.0,31,31705.0,1.0,1.0,...,NaN,0.0,0.0,NaN,0.5,NaN,25381313,NaN,NaN,2025-10-25


In [13]:
def process_riverside(raw_riverside):
    
    # extract only valid transaction types (values not in the list are the meta data rows)
    raw_riverside = raw_riverside[raw_riverside["Transaction Type"].isin(["114 - Stored ride card", "115 - Period pass", "119 - Got fare", 
                                                       "129 - Special card", "212 - Mobile Ticket Transaction"])]

    raw_riverside["date"] = pd.to_datetime(raw_riverside["Date Time"], errors="coerce").dt.date
    raw_riverside["date"] = pd.to_datetime(raw_riverside["date"], errors="coerce")
    raw_riverside["Route"] = raw_riverside["Route"].astype(int)
    raw_riverside["Stop ID"] = raw_riverside["Stop ID"].astype(int)
    
    raw_riverside_export = raw_riverside.groupby(["date", "Stop ID", "Route", "Direction"], as_index=False).agg(row_count=("Stop ID", "count"),
                                                                                     min_lon=("Longitude", lambda x: x.min()),
                                                                                     max_lat=("Latitude", lambda x: x.max()))

    return raw_riverside_export

In [29]:
input_folder = "transit_agency_ridership_raw_datasets/Riverside Transit/"
all_raw_dfs = []
all_processed_dfs = []
stop_outliers = []


for filename in os.listdir(input_folder):
    
    if filename.lower().endswith(".csv"):
        input_path = os.path.join(input_folder, filename)
        print(input_path)
        t_raw_df = pd.read_csv(input_path, header=4)
        stop_outliers.append(t_raw_df[(t_raw_df["Stop ID"] > 5000) | (t_raw_df["Stop ID"] < 1000)])
        all_raw_dfs.append(t_raw_df)
        # print(t_raw_df.columns)
        
        t_df = process_riverside(t_raw_df)
        all_processed_dfs.append(t_df)

stop_outliers = pd.concat(stop_outliers, ignore_index=True)
raw_riverside_export = pd.concat(all_processed_dfs, ignore_index=True)

raw_riverside_export.rename(columns={"row_count": "ridership",
                                         "min_lon": "Longitude",
                                         "max_lat": "Latitude"}, inplace=True)
raw_riverside_export["start_date"] = raw_riverside_export["date"]
raw_riverside_export["end_date"] = raw_riverside_export["date"]
raw_riverside_export["day_type"] = raw_riverside_export["date"].apply(get_day_type)
raw_riverside_export.to_excel("transit_agency_ridership_preprocessed_datasets/riverside_transit_ridership.xlsx")
raw_riverside_export.head()

transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(MAY_1_2025-JUNE_30_2025).csv
transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(SEPTEMBER_1_2025-OCTOBER_31_2025).csv


/tmp/ipykernel_261/1538999095.py:12: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  t_raw_df = pd.read_csv(input_path, header=4)


transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(MARCH_1_2025-APRIL_30_2025).csv


/tmp/ipykernel_261/1538999095.py:12: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  t_raw_df = pd.read_csv(input_path, header=4)


transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(JANUARY_1_2025-FEBRUARY_28_2025).csv


/tmp/ipykernel_261/1538999095.py:12: DtypeWarning: Columns (0,14,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  t_raw_df = pd.read_csv(input_path, header=4)


transit_agency_ridership_raw_datasets/Riverside Transit/TRANSACTION_DETAIL_REPORT_(JULY_1_2025-AUGUST_27_2025).csv


/tmp/ipykernel_261/1538999095.py:12: DtypeWarning: Columns (0,22) have mixed types. Specify dtype option on import or set low_memory=False.
  t_raw_df = pd.read_csv(input_path, header=4)


,date,Stop ID,Route,Direction,ridership,Longitude,Latitude,start_date,end_date,day_type
0,2025-05-01,0,1,Inbound,69,-117.594376,33.982152,2025-05-01,2025-05-01,weekday
1,2025-05-01,0,3,Inbound,9,-117.564320,33.881796,2025-05-01,2025-05-01,weekday
2,2025-05-01,0,3,Outbound,2,-117.554640,33.994648,2025-05-01,2025-05-01,weekday
3,2025-05-01,0,8,Inbound,3,-117.334352,33.695128,2025-05-01,2025-05-01,weekday
4,2025-05-01,0,16,Inbound,27,-117.339768,33.979044,2025-05-01,2025-05-01,weekday


In [20]:
stop_outliers.to_excel("stop_outliers.xlsx")

In [101]:
# t_df_riverside = raw_riverside_export.groupby(by=["date", "Stop ID", "Route", "Direction"]).count().reset_index()
# t_df_riverside.sort_values(by="row_count", ascending=False)

In [36]:
# # get max/min latitude/longitude for each stop
# idx_lat = raw_riverside.groupby("Stop ID")["Latitude"].apply(lambda x: x.abs().idxmax())
# idx_lon = raw_riverside.groupby("Stop ID")["Longitude"].apply(lambda x: x.abs().idxmax())

# loc_lat = raw_riverside.loc[idx_lat.values, ["Stop ID", "Latitude"]]
# loc_lon = raw_riverside.loc[idx_lon.values, ["Stop ID", "Longitude"]]
# loc_df = pd.merge(loc_lat, loc_lon, on="Stop ID")
# loc_df

## Define Staging Table Schema

In [16]:
with open("table_schema/staging_schema.yaml") as f:
    schema = yaml.safe_load(f)
target_cols = list(schema.keys())
schema

{'record_id': 'string',
 'dataset_id': 'string',
 'organization_name': 'string',
 'service_name': 'string',
 'route_id': 'string',
 'route_name': 'string',
 'direction': 'string',
 'stop_id': 'string',
 'stop_name': 'string',
 'stop_lat': 'float',
 'stop_lon': 'float',
 'avg_boardings': 'float',
 'avg_alightings': 'float',
 'avg_ridership': 'float',
 'day_type': 'string',
 'agg_basis': 'string',
 'start_date': 'string',
 'end_date': 'string'}

In [17]:
cols_indcator = ["route_id", "route_name", "direction", "stop_id", "stop_name", "stop_lat", "stop_lon", "avg_boardings", "avg_alightings", "avg_ridership"]

In [18]:
def load_yaml(file):
    # load yaml file that contains the raw dataset configurations, including column mapping, etc.
    with open(file) as f:
        return yaml.safe_load(f)

In [19]:
def generate_id(*args, len=16, sep="|"):
    """Generate short reproducible hash ID based on input string"""
    key = sep.join(str(arg) for arg in args if arg is not None).encode("utf-8")
    return hashlib.sha1(key).hexdigest()[:len].upper()

In [20]:
def process_agency(config_path):
    # load raw datasets based on the config yaml files for each agency
    
    config = load_yaml(config_path)
    
    file_path = config["file_path"]
    file_type = config["file_type"]

    print(file_path)
    
    if file_type == "csv":
        df_ridership = pd.read_csv(file_path)
    elif file_type in ["xlsx", "xls"]:
        df_ridership = pd.read_excel(file_path)
    else:
        raise ValueError(f"Unsupported file type")

    # ridership dataset
    # rename to unified schema
    df_ridership = df_ridership.rename(columns=config["column_mapping"])
    
    dim_cols = {}
    # add missing columns
    for col in target_cols:
        if col not in df_ridership.columns:
            df_ridership[col] = pd.NA
            dim_cols[col] = 0
        else:
            dim_cols[col] = 1
    dim_cols = {k+"_exists": v for k, v in dim_cols.items() if k in cols_indcator}

    # add static info
    df_ridership["agg_basis"] = config["agg_basis"]
    df_ridership["organization_name"] = config["organization_name"]
    df_ridership["service_name"] = config["service_name"]
    
    # generate reproducible id based on given string columns
    dataset_name = Path(file_path).stem
    dataset_id = generate_id(dataset_name)
    df_ridership["dataset_id"] = dataset_id
    # df_ridership = df_ridership.reset_index(drop=True)
    df_ridership["record_id"] = df_ridership.apply(lambda row: generate_id(row["dataset_id"],
                                                                           row["route_id"],
                                                                           row["route_name"],
                                                                           row["direction"],
                                                                           row["stop_id"],
                                                                           row["stop_name"],
                                                                           row["day_type"],
                                                                           row["agg_basis"],
                                                                           row["start_date"]), axis=1, result_type="expand")
           
    
    # returned dataframe should only have columns in target schema
    df_ridership = df_ridership[target_cols]

    # dimension table about datasets
    start_year = df_ridership["start_date"].min().year
    end_year = df_ridership["end_date"].max().year
    
    
    df_dataset_info = pd.DataFrame(data={
                        "dataset_id": [dataset_id],
                        "dataset_name": [dataset_name],
                        "organization_name": [config["organization_name"]],
                        "service_name": [config["service_name"]],
                        "start_year_collected": [start_year],
                        "end_year_collected": [end_year],
                        "agg_basis": [config["agg_basis"]],
                        "notes": [config["notes"]]
                        })
    
    for key, val in dim_cols.items():
        df_dataset_info[key] = val
        
    if not df_ridership.columns.is_unique:
        print(f"""Duplicates in DataFrame {config["organization_name"]}""")
        
    # print(df_dataset_info.columns)
    return df_ridership, df_dataset_info

In [21]:
config_paths = glob("agency_config/*.yaml")
config_paths = [c for c in config_paths]
print(config_paths)

['agency_config/golden_gate_park_shuttle.yaml', 'agency_config/samtrans.yaml', 'agency_config/sunline_transit.yaml', 'agency_config/santa_cruz_metro.yaml', 'agency_config/foothill_transit.yaml', 'agency_config/fresno_area_express.yaml', 'agency_config/sdmts.yaml', 'agency_config/golden_gate_transit.yaml', 'agency_config/bart.yaml', 'agency_config/big_blue_bus.yaml', 'agency_config/long_beach_transit.yaml', 'agency_config/omni_trans.yaml', 'agency_config/sbmtd.yaml', 'agency_config/gold_coast_transit.yaml', 'agency_config/sacrt_bus.yaml', 'agency_config/culver_citybus.yaml', 'agency_config/riverside_transit.yaml', 'agency_config/octa.yaml', 'agency_config/caltrain.yaml']


In [22]:
df_ridership, df_dataset_info = [], []

for config_path in config_paths:
    t_df_ridership, t_df_dataset = process_agency(config_path)
    if (not t_df_ridership.empty) and (not t_df_dataset.empty):
        df_ridership.append(t_df_ridership)
        df_dataset_info.append(t_df_dataset)
        
df_ridership = pd.concat(df_ridership, ignore_index=True)
df_dataset_info = pd.concat(df_dataset_info, ignore_index=True)
df_dataset_info.head()

transit_agency_ridership_preprocessed_datasets/golden_gate_park_shuttle_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/samtrans_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/sunline_transit_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/santa_cruz_metro_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/foothill_transit_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/fresno_area_express_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/sdmts_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/golden_gate_transit_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/bart_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/big_blue_bus_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/long_beach_transit_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/omni_trans_ridership.xlsx
transit_agency_ridership_preprocessed_datasets/sbmtd_ridership.xlsx
transit_ag

,dataset_id,dataset_name,organization_name,service_name,start_year_collected,end_year_collected,agg_basis,notes,route_id_exists,route_name_exists,direction_exists,stop_id_exists,stop_name_exists,stop_lat_exists,stop_lon_exists,avg_boardings_exists,avg_alightings_exists,avg_ridership_exists
0,189FC69D989010FD,golden_gate_park_shuttle_ridership,City and County of San Francisco,Golden Gate Park Shuttle,2024,2025,daily,Collected manually by operators. Weekday two v...,0,0,1,0,1,0,0,0,0,1
1,957BBF3AD8FC1B44,samtrans_ridership,San Mateo County Transit District,SamTrans,2025,2025,daily,APCs are not NTD-certified and has error rate ...,1,0,0,1,1,1,1,1,1,0
2,1B1BADA1E00153AF,sunline_transit_ridership,SunLine Transit Agency,SunLine Transit,2022,2025,fiscal year,,0,0,0,1,1,1,1,1,1,0
3,581003114DDAFDBE,santa_cruz_metro_ridership,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,2024,2025,custom period,Several buses without APC hardware (5 in March...,0,0,0,1,1,0,0,1,1,0
4,1CCF9080DC015EB8,foothill_transit_ridership,Foothill Transit,Foothill Transit,2024,2025,daily,None,1,0,1,1,0,1,1,1,1,0


In [23]:
df_ridership.head()

,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date
0,CEC173BF54FECCBD,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,NaN,<NA>,Blue Heron Boathouse,<NA>,<NA>,<NA>,<NA>,45,Weekday,daily,2024-07-01,2024-07-01
1,1BF770A6DC9B06BC,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,NaN,<NA>,Blue Heron Boathouse,<NA>,<NA>,<NA>,<NA>,59,Weekday,daily,2024-07-02,2024-07-02
2,19C42A2D3DD5337A,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,NaN,<NA>,Blue Heron Boathouse,<NA>,<NA>,<NA>,<NA>,74,Weekday,daily,2024-07-03,2024-07-03
3,40911F039E21320D,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,NaN,<NA>,Blue Heron Boathouse,<NA>,<NA>,<NA>,<NA>,177,Weekday,daily,2024-07-04,2024-07-04
4,BA5A97CE4B046876,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,NaN,<NA>,Blue Heron Boathouse,<NA>,<NA>,<NA>,<NA>,110,Weekday,daily,2024-07-05,2024-07-05


In [24]:
# convert columns that have more than one data types to explicit type
def to_clean_string(x):
    if pd.isna(x):
        return pd.NA
    if isinstance(x, (int, np.integer)):
        return str(x)
    if isinstance(x, (float, np.floating)):
        # convert 1.0 -> 1
        if x.is_integer():
            return str(int(x))
        else:
            return str(x)
    return str(x)

In [25]:
df_ridership["route_id"] = df_ridership["route_id"].apply(to_clean_string).astype("string")
df_ridership["stop_id"] = df_ridership["stop_id"].apply(to_clean_string).astype("string")
df_ridership["direction"] = df_ridership["direction"].apply(to_clean_string).astype("string")

for col in ["avg_boardings", "avg_alightings"]:
    df_ridership[col] = pd.to_numeric(df_ridership[col], errors="coerce")
    
for col in ["route_name", "stop_name"]:
    df_ridership[col] = df_ridership[col].astype("string")
    
df_ridership.head()

,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date
0,CEC173BF54FECCBD,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,<NA>,<NA>,Blue Heron Boathouse,<NA>,<NA>,NaN,NaN,45,Weekday,daily,2024-07-01,2024-07-01
1,1BF770A6DC9B06BC,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,<NA>,<NA>,Blue Heron Boathouse,<NA>,<NA>,NaN,NaN,59,Weekday,daily,2024-07-02,2024-07-02
2,19C42A2D3DD5337A,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,<NA>,<NA>,Blue Heron Boathouse,<NA>,<NA>,NaN,NaN,74,Weekday,daily,2024-07-03,2024-07-03
3,40911F039E21320D,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,<NA>,<NA>,Blue Heron Boathouse,<NA>,<NA>,NaN,NaN,177,Weekday,daily,2024-07-04,2024-07-04
4,BA5A97CE4B046876,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,<NA>,<NA>,<NA>,<NA>,Blue Heron Boathouse,<NA>,<NA>,NaN,NaN,110,Weekday,daily,2024-07-05,2024-07-05


In [26]:
df_dataset_info

,dataset_id,dataset_name,organization_name,service_name,start_year_collected,end_year_collected,agg_basis,notes,route_id_exists,route_name_exists,direction_exists,stop_id_exists,stop_name_exists,stop_lat_exists,stop_lon_exists,avg_boardings_exists,avg_alightings_exists,avg_ridership_exists
0,189FC69D989010FD,golden_gate_park_shuttle_ridership,City and County of San Francisco,Golden Gate Park Shuttle,2024,2025,daily,Collected manually by operators. Weekday two v...,0,0,1,0,1,0,0,0,0,1
1,957BBF3AD8FC1B44,samtrans_ridership,San Mateo County Transit District,SamTrans,2025,2025,daily,APCs are not NTD-certified and has error rate ...,1,0,0,1,1,1,1,1,1,0
2,1B1BADA1E00153AF,sunline_transit_ridership,SunLine Transit Agency,SunLine Transit,2022,2025,fiscal year,,0,0,0,1,1,1,1,1,1,0
3,581003114DDAFDBE,santa_cruz_metro_ridership,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,2024,2025,custom period,Several buses without APC hardware (5 in March...,0,0,0,1,1,0,0,1,1,0
4,1CCF9080DC015EB8,foothill_transit_ridership,Foothill Transit,Foothill Transit,2024,2025,daily,None,1,0,1,1,0,1,1,1,1,0
5,9B106785FD780293,fresno_area_express_ridership,City of Fresno,Fresno Area Express,2024,2025,daily,Dataset include most specific fixed route data...,0,0,0,1,1,0,0,1,1,0
6,923349C0D2AC9D75,sdmts_ridership,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,2024,2025,service period,Data collected from APCs. Stop IDs consistent ...,1,1,1,1,1,0,0,1,1,0
7,EB48A750664948FB,golden_gate_transit_ridership,"Golden Gate Bridge, Highway and Transportation...",Golden Gate Transit,2025,2025,daily,The dataset includes stop-level ridership and ...,1,0,1,1,1,0,0,1,1,0
8,011CF30F49575609,bart_ridership,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,2024,2025,daily,,0,0,0,0,1,0,0,1,1,0
9,395857DB9F379E73,big_blue_bus_ridership,City of Santa Monica,Big Blue Bus,2024,2025,service period,Big Blue Bus implement three service that chan...,1,1,1,1,1,1,1,1,1,0


In [27]:
df_dataset_info.to_csv("dim_dataset.csv")

In [28]:
df_ridership.to_csv("staging_stop_ridership.csv")

#### Sample Import check

In [67]:
config_path = 'agency_config/gold_coast_transit.yaml'
sample_ridership, sample_dataset_info = process_agency(config_path)
sample_ridership.head(2)

transit_agency_ridership_preprocessed_datasets/gold_coast_transit_ridership.xlsx


,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date
0,32CC38E679DC4DFD,AAB25897C65A4725,Gold Coast Transit District,Gold Coast Transit,1A,<NA>,North,4THBST2,4th & B St,34.198975,-119.179621,2,61,<NA>,Weekday,monthly,2025-05-01,2025-05-31
1,E61E34A2E3C846CF,AAB25897C65A4725,Gold Coast Transit District,Gold Coast Transit,1A,<NA>,South,4THBST1,4th & B St,34.199066,-119.179574,70,2,<NA>,Weekday,monthly,2025-05-01,2025-05-31


In [68]:
sample_dataset_info

,dataset_id,dataset_name,organization_name,service_name,start_year_collected,end_year_collected,agg_basis,notes,route_id_exists,route_name_exists,direction_exists,stop_id_exists,stop_name_exists,stop_lat_exists,stop_lon_exists,avg_boardings_exists,avg_alightings_exists,avg_ridership_exists
0,AAB25897C65A4725,gold_coast_transit_ridership,Gold Coast Transit District,Gold Coast Transit,2025,2025,monthly,"We received raw data for May and October, from...",1,0,1,1,1,1,1,1,1,0


In [69]:
# sample_dataset_info.dtypes

In [70]:
# duplication check
t_df_dup_check = sample_ridership.groupby(by=["organization_name", "service_name", "record_id"]).size().reset_index(name="count").sort_values(by="count", ascending=False)
t_df_dup_check[t_df_dup_check['count'] > 1]

,organization_name,service_name,record_id,count


In [15]:
# sample_ridership[sample_ridership["record_id"] == "1DF49E75"]

In [68]:
# sample_ridership.groupby(by=["organization_name", "record_id"]).size().reset_index(name="count").sort_values(by="count", ascending=False)

## Validation

In [25]:
t_ridership = df_ridership.groupby(["dataset_id", "organization_name", "service_name", "agg_basis"]).size().reset_index(name="count")
t_ridership

,dataset_id,organization_name,service_name,agg_basis,count
0,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,daily,18250
1,165BA8D319143274,Santa Barbara Metropolitan Transit District,Santa Barbara Metropolitan Transit District,monthly,7584
2,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,daily,6570
3,1B1BADA1E00153AF,SunLine Transit Agency,SunLine Transit,fiscal year,2290
4,2071F74D38820FC2,Sacramento Regional Transit District,Sacramento Regional Transit District Bus,custom period,4047
5,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,daily,5774
6,313575E419A203CA,City of Culver City,Culver CityBus,custom period,1162
7,395857DB9F379E73,City of Santa Monica,Big Blue Bus,service period,11163
8,3AF83FABCB03BDF4,Long Beach Transit,Long Beach Transit,fiscal year,9221
9,40F5F343F0BD5850,OmniTrans,OmniTrans,fiscal year,4832


In [26]:
len(df_ridership)

1188921

In [27]:
t_df_dup_check = df_ridership.groupby(by=["organization_name", "service_name", "record_id"]).size().reset_index(name="count").sort_values(by="count", ascending=False)
t_df_dup_check[t_df_dup_check['count'] > 1]

,organization_name,service_name,record_id,count


In [28]:
t_df_dup_check[(t_df_dup_check['count'] > 1) & (t_df_dup_check["organization_name"] == "City and County of San Francisco")]

,organization_name,service_name,record_id,count


In [29]:
# df_ridership[df_ridership["record_id"] == 'DC68DDBBFCF3C1B3']

In [30]:
df_ridership[(df_ridership["stop_id"].isna()) & (df_ridership["stop_name"].isna())]

,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date


In [46]:
df_ridership[df_ridership["organization_name"] == "Orange County Transportation Authority"]

,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date
1180627,F3295679F6A97F0D,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,1-Long Beach - San Clemente,N,1501,1501-PACIFIC COAST-DEL OBISPO,<NA>,<NA>,36,30,<NA>,weekday,daily,2025-02-04,2025-02-04
1180628,4EC2A2DDEE0B291D,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,1-Long Beach - San Clemente,N,1503,1503-PACIFIC COAST-ALCAZAR,<NA>,<NA>,3,3,<NA>,weekday,daily,2025-02-04,2025-02-04
1180629,33AD8D7B6AE051AB,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,1-Long Beach - San Clemente,N,1506,1506-PACIFIC COAST-AMBER LANTERN,<NA>,<NA>,6,7,<NA>,weekday,daily,2025-02-04,2025-02-04
1180630,D03799E9301CBAB9,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,1-Long Beach - San Clemente,N,1508,1508-PACIFIC COAST-SELVA,<NA>,<NA>,3,6,<NA>,weekday,daily,2025-02-04,2025-02-04
1180631,932FA7E9025EDB64,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,1-Long Beach - San Clemente,N,1509,1509-PACIFIC COAST-NIGUEL,<NA>,<NA>,13,6,<NA>,weekday,daily,2025-02-04,2025-02-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186396,A30C17BCF9B43341,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,91-Mission Viejo - Laguna Hills,S,8355,8355-AVD PICO-EL CAMINO REAL,<NA>,<NA>,3,29,<NA>,weekday,daily,2025-02-04,2025-02-04
1186397,284FA46A9C2E256A,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,91-Mission Viejo - Laguna Hills,S,8356,8356-CALLE DE LOS MOLINOS-AVD PICO,<NA>,<NA>,34,77,<NA>,weekday,daily,2025-02-04,2025-02-04
1186398,101155DB274631D7,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,91-Mission Viejo - Laguna Hills,S,8469,8469-COAST-34680 SURFSIDE INN,<NA>,<NA>,0,7,<NA>,weekday,daily,2025-02-04,2025-02-04
1186399,4C40ECAB9F1747B8,26406E10C753AC29,Orange County Transportation Authority,Orange County Transportation Authority,<NA>,91-Mission Viejo - Laguna Hills,S,8472,8472-AVD PICO-AVD VISTA HERMOSA,<NA>,<NA>,0,6,<NA>,weekday,daily,2025-02-04,2025-02-04


### Summary of Datasets

In [3]:
df_meta = pd.read_csv("dim_dataset.csv")
df_meta.head()

,Unnamed: 0,dataset_id,dataset_name,organization_name,service_name,start_year_collected,end_year_collected,agg_basis,notes,route_id_exists,route_name_exists,direction_exists,stop_id_exists,stop_name_exists,stop_lat_exists,stop_lon_exists,avg_boardings_exists,avg_alightings_exists,avg_ridership_exists
0,0,189FC69D989010FD,golden_gate_park_shuttle_ridership,City and County of San Francisco,Golden Gate Park Shuttle,2024,2025,daily,Collected manually by operators. Weekday two v...,0,0,0,0,1,0,0,0,0,1
1,1,957BBF3AD8FC1B44,samtrans_ridership,San Mateo County Transit District,SamTrans,2025,2025,daily,APCs are not NTD-certified and has error rate ...,1,0,0,1,1,0,0,1,1,0
2,2,1B1BADA1E00153AF,sunline_transit_ridership,SunLine Transit Agency,SunLine Transit,2022,2025,fiscal year,NaN,0,0,0,1,1,1,1,1,1,0
3,3,581003114DDAFDBE,santa_cruz_metro_ridership,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,2024,2025,custom period,Several buses without APC hardware (5 in March...,0,0,0,1,1,0,0,1,1,0
4,4,9B106785FD780293,fresno_area_express_ridership,City of Fresno,Fresno Area Express,2024,2025,daily,Dataset include most specific fixed route data...,0,0,0,1,1,0,0,1,1,0


#### Next step -> fact table

- Join unique stop id across state
- Sum up ridership for the same stop